### Step 1: Setup storage access keys

#### swap out the code snippet below to configure with you account access key to your adls gen 2 storage account

In [0]:
storage_account_name = "dbxsummitstg"
storage_account_access_key=dbutils.secrets.get('aml','mykeyforstorageacct')
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".dfs.core.windows.net",
  storage_account_access_key)

### Step 2: Get data and store in Delta file format in 'Silver Zone'

In [0]:
import os
import numpy as np
import pandas as pd


In [0]:
%sh wget https://azmlworkshopdata.blob.core.windows.net/safedriverdata/porto_seguro_safe_driver_prediction_train.csv -P /tmp

--2021-06-10 18:21:19-- https://azmlworkshopdata.blob.core.windows.net/safedriverdata/porto_seguro_safe_driver_prediction_train.csv
Resolving azmlworkshopdata.blob.core.windows.net (azmlworkshopdata.blob.core.windows.net)... 20.38.101.132
Connecting to azmlworkshopdata.blob.core.windows.net (azmlworkshopdata.blob.core.windows.net)|20.38.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115852544 (110M) [application/vnd.ms-excel]
Saving to: ‘/tmp/porto_seguro_safe_driver_prediction_train.csv’

 0K .......... .......... .......... .......... .......... 0% 105M 1s
 50K .......... .......... .......... .......... .......... 0% 148M 1s
 100K .......... .......... .......... .......... .......... 0% 143M 1s
 150K .......... .......... .......... .......... .......... 0% 126M 1s
 200K .......... .......... .......... .......... .......... 0% 198M 1s
 250K .......... .......... .......... .......... .......... 0% 90.1M 1s
 300K .......... .......... .......... .......... .......... 0% 199M 1s
 350K .......... .......... .......... .......... .......... 0% 19.4M 1s
 400K .......... .......... .......... .......... .......... 0% 129M 1s
 450K .......... .......... .......... .......... .......... 0% 222M 1s
 500K .......... .......... .......... .......... .......... 0% 215M 1s
 550K .......... .......... .......... .......... .......... 0% 107M 1s
 600K .......... .......... .......... .......... .......... 0% 150M 1s
 650K .......... .......... .......... .......... .......... 0% 84.0M 1s
 700K .......... .......... .......... .......... .......... 0% 90.4M 1s
 750K .......... .......... .......... .......... .......... 0% 61.4M 1s
 800K .......... .......... .......... .......... .......... 0% 73.8M 1s
 850K .......... .......... .......... .......... .......... 0% 74.5M 1s
 900K .......... .......... .......... .......... .......... 0% 22.1M 1s
 950K .......... .......... .......... .......... .......... 0% 66.5M 1s
 1000K .......... .......... .......... .......... .......... 0% 72.3M 1s
 1050K .......... .......... .......... .......... .......... 0% 73.3M 1s
 1100K .......... .......... .......... .......... .......... 1% 76.4M 1s
 1150K .......... .......... .......... .......... .......... 1% 62.5M 1s
 1200K .......... .......... .......... .......... .......... 1% 80.9M 1s
 1250K .......... .......... .......... .......... .......... 1% 149M 1s
 1300K .......... .......... .......... .......... .......... 1% 199M 1s
 1350K .......... .......... .......... .......... .......... 1% 174M 1s
 1400K .......... .......... .......... .......... .......... 1% 12.7M 2s
 1450K .......... .......... .......... .......... .......... 1% 194M 2s
 1500K .......... .......... .......... .......... .......... 1% 185M 2s
 1550K .......... .......... .......... .......... .......... 1% 169M 2s
 1600K .......... .......... .......... .......... .......... 1% 168M 1s
 1650K .......... .......... .......... .......... .......... 1% 197M 1s
 1700K .......... .......... .......... .......... .......... 1% 201M 1s
 1750K .......... .......... .......... .......... .......... 1% 178M 1s
 1800K .......... .......... .......... .......... .......... 1% 195M 1s
 1850K .......... .......... .......... .......... .......... 1% 208M 1s
 1900K .......... .......... .......... .......... .......... 1% 186M 1s
 1950K .......... .......... .......... .......... .......... 1% 172M 1s
 2000K .......... .......... .......... .......... .......... 1% 206M 1s
 2050K .......... .......... .......... .......... .......... 1% 27.4M 1s
 2100K .......... .......... .......... .......... .......... 1% 87.1M 1s
 2150K .......... .......... .......... .......... .......... 1% 146M 1s
 2200K .......... .......... .......... .......... .......... 1% 201M 1s
 2250K .......... .......... .......... .......... .......... 2% 178M 1s
 2300K .......... .......... .......... .......... .......... 2% 172M 1s
 2350K .......... .......... .......... .....

In [0]:
# Directly load from file into Pandas DataFrame
DATA_DIR = "/tmp"
pandas_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_train.csv'))

print(pandas_df.shape)
pandas_df.head(5)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [0]:
safedriverdata_df = spark.createDataFrame(pandas_df)

# Add the spark data frame to the catalog
safedriverdata_df.createOrReplaceTempView('safedriverdata')

In [0]:
%sql
select * from safedriverdata

id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.7180703307999999,10,1,-1,0,1,4,1,0,0,1,12,2,0.4,0.8836789178,0.3708099244,3.6055512755,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.7660776723,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316227766,0.6188165191,0.3887158345,2.4494897428,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.0,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316227766,0.6415857163,0.34727510710000004,3.3166247904,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.5809475019,7,1,0,0,1,11,1,1,3,1,104,1,0.3741657387,0.5429487899000001,0.2949576241,2.0,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840758586,11,1,-1,0,-1,14,1,1,2,1,82,3,0.3160696126,0.5658315025,0.3651027253,2.0,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
19,0,5,1,4,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,1.8,2.3326487091,10,0,-1,0,0,14,1,1,0,1,104,2,0.4459820624000001,0.879049073,0.4062019202,3.0,0.7,0.8,0.4,3,1,8,2,11,3,8,4,2,0,9,0,1,0,1,1,1
20,0,2,1,3,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.6,0.1,0.6174544518,6,1,-1,0,1,11,1,1,0,1,99,2,0.316227766,0.6396829018,0.3687817783,3.1622776602,0.2,0.6,0.5,2,2,8,1,8,3,10,3,0,0,10,0,1,0,0,1,0
22,0,5,1,4,0,0,1,0,0,0,0,0,0,0,0,13,1,0,0,0.7,0.4,0.6072478901,11,1,0,0,0,11,1,1,2,1,30,3,0.4469899328,0.9005737382,0.3748332963,3.3166247904,0.1,0.5,0.1,1,2,7,1,6,1,13,7,1,3,6,1,0,1,0,1,0
26,0,5,1,3,1,0,0,0,1,0,0,0,0,0,0,6,1,0,0,0.9,0.7,0.9013878189,10,1,-1,0,1,14,1,1,0,1,68,3,0.4,0.7806412918000001,0.4074309757,2.8284271247,0.9,0.8,0.6,3,1,7,3,9,4,11,4,2,1,5,0,1,0,0,0,1
28,1,1,1,2,0,0,0,1,0,0,0,0,0,0,0,4,0,0,1,0.9,1.4,2.3166516786,11,0,0,1,0,14,1,1,2,1,104,2,0.4472135955,1.4581843686000002,0.390256326,3.6055512755,0.7,0.8,0.8,2,2,8,2,9,1,11,3,5,0,6,0,1,0,0,1,0


#### swap out for your storage account path:

In [0]:
delta_silver_path= "abfss://silver@dbxsummitstg.dfs.core.windows.net/safedriverdata_delta"
#safedriverdata_df.write.mode('overwrite').delta(delta_silver)
safedriverdata_df.write.mode("overwrite").format("delta").save(delta_silver_path)

### Step 3: Register Parquet data in AML Datastore/Dataset using SPN

#### Swap out for your AML configuration

In [0]:
from azureml.core.authentication import ServicePrincipalAuthentication

#svcdbxsummit service principle
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
app_id="b13e47be-a9b6-4431-8e29-9b32bf505206"
svc_pr_password=dbutils.secrets.get('aml','mykeyforsp')

#ServicePrincipalAuthentication(tenant_id, service_principal_id, service_principal_password, cloud='AzureCloud', _enable_caching=True)
svc_pr = ServicePrincipalAuthentication(
       tenant_id=tenant_id,
       service_principal_id=app_id,
       service_principal_password=svc_pr_password) 

In [0]:
from azureml.core import Workspace, Dataset

subscription_id = "5763fde3-4253-480c-928f-dfe1e8888a57"  #you should be owner or contributor
resource_group = "dbx_tech_summit" #you should be owner or contributor
workspace_name = "aml_dbx_summit"
workspace_region = "eastus2"

ws = Workspace.get(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group,
               auth=svc_pr)

#### Either get existing datastore in cell below or register a new datastore (2 cells below)

In [0]:
from azureml.core.datastore import Datastore
# Get a named datastore from the current workspace
datastore = Datastore.get(ws, datastore_name='safedriverdatastore')

```
from azureml.core.datastore import Datastore
safedriver_datastore= "safedriverdatastore"
file_system = "silver"
# register a new datastore in the workspace by name
mydatastore = Datastore.register_azure_data_lake_gen2(ws, safedriver_datastore, file_system, storage_account_name, tenant_id, app_id, svc_pr_password, resource_url=None, authority_url=None, protocol=None, endpoint=None, overwrite=False)
```

In [0]:
# create file dataset from all jpeg files in the directory
file_dataset = Dataset.File.from_files(path=(datastore,'safedriverdata_delta/**/*.*'))
file_dataset.register(workspace = ws,
           name = 'delta_safedriverdataset',
           description = 'safedriver training data in delta file format',
           create_new_version = True)

Out[15]: {
 "source": [
 "('safedriverdatastore', 'safedriverdata_delta/**/*.*')"
 ],
 "definition": [
 "GetDatastoreFiles"
 ],
 "registration": {
 "id": "b90c857c-9526-41a7-ace5-447b8e9e21ec",
 "name": "delta_safedriverdataset",
 "version": 1,
 "description": "safedriver training data in delta file format",
 "workspace": "Workspace.create(name='aml_dbx_summit', subscription_id='5763fde3-4253-480c-928f-dfe1e8888a57', resource_group='dbx_tech_summit')"
 }
}

### Step 4: Validate dataset from AML

In [0]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(workspace = ws, name='delta_safedriverdataset')
dataset.download(target_path='./delta_driver', overwrite=False)

Out[11]: ['/databricks/driver/delta_driver/_delta_log/00000000000000000000.crc',
 '/databricks/driver/delta_driver/_delta_log/00000000000000000000.json',
 '/databricks/driver/delta_driver/part-00000-6457272b-ca2d-495b-8339-bd52b93ed79e-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00001-0b71797c-c968-4aee-a111-88b3e54fd36d-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00002-b3b60210-f2c8-4b02-bbc3-c290556f83b6-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00003-d90dfe34-5f2c-4428-a49c-ae056d737e7e-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00004-d22b1ec0-1224-4d72-9fd7-7c3d3d175a2c-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00005-ad9dee32-9d9e-48ae-b514-888e9bcda997-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00006-6c65e651-2efb-4150-8d4d-d572213b20a5-c000.snappy.parquet',
 '/databricks/driver/delta_driver/part-00007-59ac8142-44a8-4073-ad49-75cadab882d2-c000.snappy.parquet']